In [12]:
from google.colab import files
uploaded = files.upload()

Saving lfw-funneled.tgz.zip to lfw-funneled.tgz.zip


In [14]:
# 1. Dézipper le fichier .zip
!unzip lfw-funneled.tgz.zip

# 2. Extraire le .tgz (archive tar gzip)
!tar -xvzf lfw-funneled.tgz

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
lfw_funneled/Guillaume_Depardieu/Guillaume_Depardieu_0001.jpg
lfw_funneled/Delphine_Chuillot/
lfw_funneled/Delphine_Chuillot/Delphine_Chuillot_0001.jpg
lfw_funneled/Paul_LeClerc/
lfw_funneled/Paul_LeClerc/Paul_LeClerc_0001.jpg
lfw_funneled/Eddie_Jordan/
lfw_funneled/Eddie_Jordan/Eddie_Jordan_0001.jpg
lfw_funneled/Lionel_Richie/
lfw_funneled/Lionel_Richie/Lionel_Richie_0001.jpg
lfw_funneled/Lionel_Richie/Lionel_Richie_0002.jpg
lfw_funneled/Jose_Viegas_Filho/
lfw_funneled/Jose_Viegas_Filho/Jose_Viegas_Filho_0001.jpg
lfw_funneled/Jose_Viegas_Filho/Jose_Viegas_Filho_0002.jpg
lfw_funneled/Roger_Mahony/
lfw_funneled/Roger_Mahony/Roger_Mahony_0001.jpg
lfw_funneled/Clay_Aiken/
lfw_funneled/Clay_Aiken/Clay_Aiken_0001.jpg
lfw_funneled/Clay_Aiken/Clay_Aiken_0002.jpg
lfw_funneled/Clay_Aiken/Clay_Aiken_0003.jpg
lfw_funneled/Clay_Aiken/Clay_Aiken_0004.jpg
lfw_funneled/Clay_Aiken/Clay_Aiken_0005.jpg
lfw_funneled/Clay_Aiken/

In [ ]:
# 1. Décompression (uniquement si besoin, par ex. sur Colab)
# !unzip lfw-funneled.tgz.zip
# !tar -xvzf lfw-funneled.tgz

# 2. Imports
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from collections import Counter

# 3. Variables
data_dir = 'lfw_funneled'  # dossier extrait
img_size = 64

# 4. Chargement images + labels
images = []
labels = []

for person_name in os.listdir(data_dir):
    person_folder = os.path.join(data_dir, person_name)
    if os.path.isdir(person_folder):
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            try:
                img = Image.open(image_path).convert('RGB').resize((img_size, img_size))
                images.append(np.array(img))
                labels.append(person_name)
            except:
                print(f"Impossible de charger {image_path}")

print(f"Images chargées : {len(images)}")

# 5. Filtrage pour garder uniquement les personnes avec au moins 2 images
counts = Counter(labels)
filtered_images = []
filtered_labels = []

for img, lbl in zip(images, labels):
    if counts[lbl] >= 2:
        filtered_images.append(img)
        filtered_labels.append(lbl)

print(f"Images après filtrage : {len(filtered_images)}")

# 6. Préparation des données
X = np.array(filtered_images) / 255.0
le = LabelEncoder()
y_enc = le.fit_transform(filtered_labels)
y = to_categorical(y_enc)

# 7. Split train/test avec stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y_enc
)

# 8. Création du modèle CNN simple
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(y.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 9. Entraînement (5 epochs, batch size 32)
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

# 10. Évaluation
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")


Images chargées : 13233
Images après filtrage : 9164


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 44s 202ms/step - accuracy: 0.0512 - loss: 7.1039 - val_accuracy: 0.0613 - val_loss: 6.7879
Epoch 2/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 37s 181ms/step - accuracy: 0.0591 - loss: 6.5345 - val_accuracy: 0.0777 - val_loss: 6.7177
Epoch 3/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 40s 176ms/step - accuracy: 0.0758 - loss: 6.1403 - val_accuracy: 0.0926 - val_loss: 6.5594
Epoch 4/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 40s 173ms/step - accuracy: 0.1087 - loss: 5.6140 - val_accuracy: 0.1008 - val_loss: 6.4359
Epoch 5/5
207/207 ━━━━━━━━━━━━━━━━━━━━ 40s 168ms/step - accuracy: 0.1595 - loss: 4.9956 - val_accuracy: 0.1308 - val_loss: 6.8753
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.1343 - loss: 5.7041
Test Accuracy: 0.15


In [ ]:
import os


In [ ]:
from PIL import Image
import numpy as np


In [ ]:
# 5.1 Filtrage : seulement les personnes avec ≥ 10 images
min_images = 10
filtered_images = []
filtered_labels = []

for img, lbl in zip(images, labels):
    if counts[lbl] >= min_images:
        filtered_images.append(img)
        filtered_labels.append(lbl)

print(f"Images conservées (≥{min_images} par personne) : {len(filtered_images)}")


Images conservées (≥10 par personne) : 0


In [ ]:
from collections import Counter

min_images = 2  # on garde les personnes ayant AU MOINS 2 images

counts = Counter(labels)
filtered_images = []
filtered_labels = []

for img, lbl in zip(images, labels):
    if counts[lbl] >= min_images:
        filtered_images.append(img)
        filtered_labels.append(lbl)

print(f"Images conservées (≥{min_images} par personne) : {len(filtered_images)}")


Images conservées (≥2 par personne) : 0


In [17]:
from collections import Counter

min_images = 3  # on garde les personnes ayant AU MOINS 2 images

counts = Counter(labels)
filtered_images = []
filtered_labels = []

for img, lbl in zip(images, labels):
    if counts[lbl] >= min_images:
        filtered_images.append(img)
        filtered_labels.append(lbl)

print(f"Images conservées (≥{min_images} par personne) : {len(filtered_images)}")

Images conservées (≥3 par personne) : 7606


In [19]:
# Nombre de classes dans y_train
num_classes_train = len(np.unique(np.argmax(y_train, axis=1)))
val_size = 0.1
min_val_samples = int(val_size * len(X_train))

if min_val_samples < num_classes_train:
    val_size = num_classes_train / len(X_train)
    print(f"Ajustement taille validation à {val_size:.3f} pour avoir au moins un exemple par classe.")

X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train, test_size=val_size, random_state=42, stratify=np.argmax(y_train, axis=1)
)


Ajustement taille validation à 0.148 pour avoir au moins un exemple par classe.


In [20]:
# 1. Décompression (si nécessaire sur Colab)
# Décommente ces lignes si tu n'as pas encore décompressé
# !unzip lfw-funneled.tgz.zip
# !tar -xvzf lfw-funneled.tgz

# 2. Imports
import os
import numpy as np
from PIL import Image
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# 3. Variables
data_dir = 'lfw_funneled'
img_size = 64

# 4. Chargement des images
images = []
labels = []

for person_name in os.listdir(data_dir):
    person_folder = os.path.join(data_dir, person_name)
    if os.path.isdir(person_folder):
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            try:
                img = Image.open(image_path).convert('RGB').resize((img_size, img_size))
                images.append(np.array(img))
                labels.append(person_name)
            except:
                print(f"Impossible de charger {image_path}")

print(f"Images chargées : {len(images)}")

# 5. Filtrage : seulement personnes avec ≥3 images
counts = Counter(labels)
filtered_images = []
filtered_labels = []

min_images = 3
for img, lbl in zip(images, labels):
    if counts[lbl] >= min_images:
        filtered_images.append(img)
        filtered_labels.append(lbl)

print(f"Images conservées (≥{min_images} par personne) : {len(filtered_images)}")

# 6. Préparation des données
X = np.array(filtered_images) / 255.0
le = LabelEncoder()
y_enc = le.fit_transform(filtered_labels)
y = to_categorical(y_enc)

# 7. Split train/test (20%) avec stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y_enc
)

# 8. Split train/val (10% du train) – on ajuste pour éviter erreur
stratify_labels = np.argmax(y_train, axis=1)
val_size = max(0.148, len(np.unique(stratify_labels)) / len(y_train))  # ~14.8% ou + si nécessaire
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train, test_size=val_size, random_state=42, stratify=stratify_labels
)

# 9. Augmentation de données
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train_final)

# 10. Modèle CNN amélioré
model = Sequential([
    Input(shape=(img_size, img_size, 3)),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.3),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(y.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 11. Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 12. Entraînement
history = model.fit(
    datagen.flow(X_train_final, y_train_final, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=25,
    callbacks=[early_stop]
)

# 13. Évaluation finale
loss, acc = model.evaluate(X_test, y_test)
print(f"\n✅ Test Accuracy: {acc:.2%}")


Images chargées : 13233
Images conservées (≥3 par personne) : 7606
Epoch 1/25


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


162/162 ━━━━━━━━━━━━━━━━━━━━ 55s 320ms/step - accuracy: 0.0612 - loss: 6.4873 - val_accuracy: 0.0699 - val_loss: 6.0675
Epoch 2/25
162/162 ━━━━━━━━━━━━━━━━━━━━ 51s 316ms/step - accuracy: 0.0711 - loss: 6.1635 - val_accuracy: 0.0699 - val_loss: 5.9894
Epoch 3/25
162/162 ━━━━━━━━━━━━━━━━━━━━ 46s 281ms/step - accuracy: 0.0725 - loss: 6.0252 - val_accuracy: 0.0733 - val_loss: 5.8982
Epoch 4/25
162/162 ━━━━━━━━━━━━━━━━━━━━ 44s 273ms/step - accuracy: 0.0709 - loss: 5.9392 - val_accuracy: 0.0744 - val_loss: 5.7803
Epoch 5/25
162/162 ━━━━━━━━━━━━━━━━━━━━ 48s 294ms/step - accuracy: 0.0739 - loss: 5.8079 - val_accuracy: 0.0766 - val_loss: 5.6250
Epoch 6/25
162/162 ━━━━━━━━━━━━━━━━━━━━ 45s 280ms/step - accuracy: 0.0760 - loss: 5.6872 - val_accuracy: 0.0899 - val_loss: 5.5322
Epoch 7/25
162/162 ━━━━━━━━━━━━━━━━━━━━ 48s 293ms/step - accuracy: 0.0887 - loss: 5.4970 - val_accuracy: 0.1021 - val_loss: 5.3967
Epoch 8/25
162/162 ━━━━━━━━━━━━━━━━━━━━ 45s 276ms/step - accuracy: 0.0956 - loss: 5.3827 - val

In [21]:
# 1. Décompression (à exécuter une seule fois si nécessaire sur Colab)
# !unzip lfw-funneled.tgz.zip
# !tar -xvzf lfw-funneled.tgz

# 2. Imports
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from collections import Counter

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# 3. Paramètres
data_dir = 'lfw_funneled'
img_size = 160  # MobileNetV2 input size
min_images = 15  # Nombre minimum d’images par personne

# 4. Chargement des images et labels
images = []
labels = []

for person_name in os.listdir(data_dir):
    person_folder = os.path.join(data_dir, person_name)
    if os.path.isdir(person_folder):
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            try:
                img = Image.open(image_path).convert('RGB').resize((img_size, img_size))
                images.append(np.array(img))
                labels.append(person_name)
            except:
                print(f"Erreur lors du chargement de {image_path}")

print(f"Images chargées : {len(images)}")

# 5. Filtrage : personnes avec suffisamment d'images
counts = Counter(labels)
filtered_images = []
filtered_labels = []

for img, lbl in zip(images, labels):
    if counts[lbl] >= min_images:
        filtered_images.append(img)
        filtered_labels.append(lbl)

print(f"Images conservées (≥{min_images} images/pers.) : {len(filtered_images)}")

# 6. Préparation des données
X = np.array(filtered_images) / 255.0
le = LabelEncoder()
y_enc = le.fit_transform(filtered_labels)
y = to_categorical(y_enc)

# 7. Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y_enc
)

# 8. Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

# 9. Base MobileNetV2 (couches gelées)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_size, img_size, 3)))
base_model.trainable = False  # On gèle les couches pré-entraînées

# 10. Ajout du classifieur personnalisé
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(y.shape[1], activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# 11. Compilation
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 12. Entraînement
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=20,
    callbacks=[early_stop]
)

# 13. Évaluation finale
loss, acc = model.evaluate(X_test, y_test)
print(f"\n✅ Test Accuracy: {acc * 100:.2f}%")


Images chargées : 13233
Images conservées (≥15 images/pers.) : 3595


/tmp/ipython-input-3391934869.py:78: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_size, img_size, 3)))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 109s 1s/step - accuracy: 0.1055 - loss: 4.4604 - val_accuracy: 0.1947 - val_loss: 3.5908
Epoch 2/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.1848 - loss: 3.6883 - val_accuracy: 0.2865 - val_loss: 3.1902
Epoch 3/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 87s 963ms/step - accuracy: 0.2450 - loss: 3.2979 - val_accuracy: 0.3310 - val_loss: 2.9585
Epoch 4/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.2774 - loss: 3.1100 - val_accuracy: 0.3574 - val_loss: 2.6826
Epoch 5/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 91s 1s/step - accuracy: 0.3058 - loss: 2.8250 - val_accuracy: 0.3797 - val_loss: 2.5472
Epoch 6/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.3180 - loss: 2.7593 - val_accuracy: 0.4019 - val_loss: 2.4638
Epoch 7/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.3338 - loss: 2.5974 - val_accuracy: 0.4186 - val_loss: 2.3534
Epoch 8/20
90/90 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.3743 - loss: 2.4197 - val_accuracy: 0.4395 - val_l